In [485]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import tensorflow as tf
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")
import contextlib
import _pickle as cPickle
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import ast
import pickle
import copy
import sys
sys.path.append('C:\\Users\\ae2722\\Documents\\DCI_code\\helperCode')
import main_create_model as mm
from matplotlib.cbook import boxplot_stats
from matplotlib.pyplot import figure

## Feature importances

In [112]:
path = '/Users/ae2722/Documents/DCI_code/'
numofhours = 24
run = 100
client = 'Aachen'

In [85]:
##load the data and create train/test split
def dataloader_used(path, client, numofhours):
    dfs = ['X_train_norm', 'X_test_norm', 'X_val_norm', 'y_train', 'y_test', 'y_val']
    for i, df in zip(range(6), dfs):
        globals()[df] = pd.read_excel(open(f'{path}client/Data_subpopn/dataset_client_{client}_{numofhours}.xlsx', 'rb'),
                  sheet_name=f'sheet{i}', index_col = 0)
    return X_train_norm, X_test_norm, X_val_norm, y_train, y_test, y_val

In [86]:
def dataloader_full(client, path, numofhours):
    ##load the data and create train/test split
    keys = ['HR', 'AR-M', 'AR-D', 'AR-S', 'SPO2', 'RR']
    target = 'label'
    IDcol = 'Shopid'
    hourscol = 'hours'
    start_time = 24*3
    k_feat = 70

    all_predictors, labelsvals, demfeats_vals = mm.load_data_all(univ_type=client,include_dems = True)
        
    predictor_keys = mm.get_predictor(keys)
    demo_predictor_keys = mm.get_dem_predictors() 

    continuous_cols = np.array(predictor_keys)
    continuous_cols = np.append(continuous_cols, np.array(demo_predictor_keys)[[0, 5]])
    categorical_cols = np.array(demo_predictor_keys)[[1,2,3,4]]

    X, y = mm.create_data_input(all_predictors, start_time, IDcol, predictor_keys, demfeats_vals, labelsvals, numofhours)
    return X, y

In [27]:
##LOAD LR
def load_LR_weights(path, client, numofhours, run):
    model = tf.keras.models.load_model(f'{path}server/model_full/server_models/best_model/LR/client_{client}_{numofhours}_{run}.h5')
    weights = model.layers[0].get_weights()[0].reshape(1,-1)[0]
    return weights

In [8]:
##load SVM
def load_SVM_model(path, client, numofhours, run):
    with open(f'{path}client/model/SVM/SVM_{client}_{numofhours}_{run}.pkl', 'rb') as f:
        return  cPickle.load(f)

In [28]:
##load the rf  model
def load_RF_feature_imp(path, client, numofhours, run):
    with open(f'{path}server/model_full/server_models/best_model/RF/RF_{client}_{numofhours}_{run}.pkl', 'rb') as f:
        model =  cPickle.load(f)
    return model.feature_importances_

In [43]:
def add_to_dict(features, model):
    for m in model:
        if m in features:
            features[m] +=1
        else:
            features[m] = 1
    return features

In [102]:
def get_imp_features(path, client):
    numofhours_all = [i for i in range(24,157,12)]
    features_all = {}
    X,y = dataloader_full(client, path, 24)
    for numofhours in numofhours_all:
        features = {}
        for i in range(300):
            try:
                lr = load_LR_weights(path, client, numofhours, i)
                rf = load_RF_feature_imp(path, client, numofhours, i)
                lr_3 = np.argsort(abs(lr))[-5:][::-1]
                rf_3 = np.argsort(abs(rf))[-5:][::-1]
                features = add_to_dict(features, lr_3)
                features = add_to_dict(features, rf_3)
            except:
                pass
        counts = pd.DataFrame.from_dict(features, orient = 'index', columns = ['count']).sort_values(by = 'count',ascending = False)
        columns_imp = counts.iloc[:5].index
        X_train_norm, X_test_norm, X_val_norm, y_train, y_test, y_val = dataloader_used(path, client, numofhours)
        column_index = X_train_norm.iloc[:,columns_imp].columns
        features = list(X.iloc[:, column_index].columns)
        features_all[numofhours] = features
    return features_all

In [107]:
features_all = get_imp_features(path, 'Aachen')

In [111]:
df_aachen = pd.DataFrame.from_dict(features_all, orient = 'index')
df_aachen

,0,1,2,3,4
24,"(Mean_HR, entropy_)","(Mean_min_Xcorr_HR_SPO2, entropy_)","(Mean_SPO2, nanmean)","(Mean_min_Xcorr_HR_RR, maxminRange_)","(Mean_max_Xcorr_HR_SPO2, entropy_)"
36,"(Mean_HR, entropy_)","(Mean_min_Xcorr_HR_SPO2, entropy_)","(Mean_min_Xcorr_HR_RR, maxminRange_)","(Mean_SPO2, entropy_)","(Mean_SPO2, nanmean)"
48,"(Mean_HR, entropy_)","(Mean_min_Xcorr_HR_SPO2, entropy_)","(Mean_SPO2, nanmean)","(STD_SPO2, maxminRange_)","(Mean_max_Xcorr_HR_AR-M, entropy_)"
60,"(Mean_HR, entropy_)","(Mean_min_Xcorr_HR_SPO2, entropy_)","(STD_SPO2, entropy_)","(Mean_SPO2, nanmean)","(STD_HR, nanmean)"
72,"(Mean_HR, entropy_)","(Mean_min_Xcorr_HR_SPO2, entropy_)","(STD_HR, nanmean)","(STD_SPO2, entropy_)","(Mean_SPO2, nanmean)"
84,"(Mean_HR, entropy_)","(Mean_min_Xcorr_HR_SPO2, entropy_)","(Mean_SPO2, nanmean)","(STD_HR, maxminRange_)","(STD_SPO2, entropy_)"
96,"(Mean_min_Xcorr_HR_SPO2, entropy_)","(STD_HR, nanmean)","(Mean_HR, entropy_)","(Mean_max_Xcorr_HR_RR, entropy_)","(Mean_max_Xcorr_HR_SPO2, nanstd)"
108,"(Mean_min_Xcorr_HR_SPO2, entropy_)","(Mean_HR, entropy_)","(Mean_max_Xcorr_HR_RR, entropy_)","(STD_HR, nanmean)","(Mean_SPO2, nanmean)"
120,"(Mean_min_Xcorr_HR_SPO2, entropy_)","(Mean_max_Xcorr_HR_RR, entropy_)","(Mean_HR, entropy_)","(STD_HR, nanmean)","(Mean_max_Xcorr_HR_AR-D, nanmedian)"
132,"(Mean_min_Xcorr_HR_SPO2, entropy_)","(Mean_max_Xcorr_HR_RR, entropy_)","(STD_HR, IQR_Range)","(Mean_max_Xcorr_HR_AR-D, nanmedian)","(Mean_max_Xcorr_HR_AR-D, nanmean)"


## Feature distributions

In [138]:
features_plot = {}
outcomes = {}
numofhours_all = [i for i in range(24,157,12)]
clients = ['CUMC', 'UTH', 'Aachen']
feat = pd.unique(df_aachen.values.ravel('K'))
for client in clients:
    features_plot[client] = {}
    for numofhours in numofhours_all:
        X, y = dataloader_full(client, path, numofhours)
        features_plot[client][numofhours] = X.loc[:,feat]
        outcomes[client] = y


In [276]:
plot_dfs = {}
feats = pd.unique(df_aachen.values.ravel('K'))
for f in feats:
    plot_dfs[f] = {}
    for client in clients:
        df = pd.DataFrame()
        for numofhours in numofhours_all:
            feat_ = features_plot[client][numofhours].loc[:,f]
            feat_ = pd.DataFrame(feat_)
            feat_.columns = [f"{numofhours}_{pair[0]}_{pair[1]}" for pair in feat_.columns]
            df = pd.concat([df,feat_], axis = 1)
        df_T = df.T
        df_T.index = numofhours_all
        plot_dfs[f][client] =  df_T

In [375]:
with open(f"{path}server/results/feature_distributions.pkl", 'wb') as handle:
    #pickle.dump(plot_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f"{path}server/results/outcomes.pkl", 'wb') as handle:
    pickle.dump(outcomes, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [174]:
with open(f"{path}server/results/feature_distributions.pkl", 'rb') as handle:
    a = pickle.load(handle)
    
with open(f"{path}server/results/outcomes.pkl", 'rb') as handle:
    b = pickle.load(handle)

In [568]:
def grapher_box(f, plot_dfs, group, hours):
    figure(figsize = (15, 10), dpi = 80)
    graph = pd.DataFrame()
    for client in clients:
        groups = {1.0:'DCI+', 0.0:'DCI-', 'all':'all'}
        
        df = plot_dfs[f][client]
        df = df.loc[hours]
        if group != 'all':
            df_ = df.loc[:,outcomes[client].index[outcomes[client]['label'] == group]]
        else:
            df_ = df 
        boxplot_stats(df)
        data = df_.reset_index().melt(id_vars = 'index')
        data.drop('Shopid', axis = 1, inplace = True)
        data['site'] = client
        outliers = boxplot_stats(data['value'])[0]['fliers']
        data = data[~data['value'].isin(outliers)]
        graph = pd.concat([graph, data])
    
        
    fig, ax = plt.subplots(figsize = (12,10))
    sns.boxplot(x = graph['index'], y = graph['value'], hue = graph['site'], fliersize = 0, ax = ax)
    x = f[0].strip('(').strip(')')
    x = x + f'_{f[1]}'
    
    plt.title(f'Distribution_for_{x}_{groups[group]}')
    plt.xlabel('Hours from DCI')
    plt.xticks(ticks = hours, labels = hours)
    plt.ylabel('Value')
    plt.legend(bbox_to_anchor = (1,1))
    plt.plot()
    return

In [526]:
def grapher_line(f, plot_dfs, group, hours):
    figure(figsize = (15, 10), dpi = 80)
    graph = pd.DataFrame()
    for client in clients:
        groups = {1.0:'DCI+', 0.0:'DCI-', 'all':'all'}
        
        df = plot_dfs[f][client]
        df = df.loc[hours]
        if group != 'all':
            df_ = df.loc[:,outcomes[client].index[outcomes[client]['label'] == group]]
        else:
            df_ = df 
        
        data = (pd.DataFrame(df_.mean(axis = 1), columns = [client]))
        graph = pd.concat([graph, data])
    
        
        
        '''
        sns.set_theme('white')
        y_errormin = df_.apply(lambda x: np.percentile(x, 50), axis = 1)  - df_.apply(lambda x: np.percentile(x, 49), axis = 1) 
        y_errormax = df_.apply(lambda x: np.percentile(x, 80), axis = 1) - df_.apply(lambda x: np.percentile(x, 50), axis = 1) 
        y_error = np.array([y_errormin, y_errormax])
        plt.errorbar(df_.index,df_.apply(lambda x: np.percentile(x, 50), axis = 1),
                     yerr = y_error,
                    label = client,
                    alpha = 0.7,
                    linestyle = '-')
        '''
    fig, ax = plt.subplots(figsize = (12,10))
    sns.lineplot(data = graph, ax = ax)
    x = f[0].strip('(').strip(')')
    x = x + f'_{f[1]}'
    
    plt.title(f'Distribution_for_{x}_{groups[group]}')
    plt.xlabel('Hours from DCI')
    plt.xticks(ticks = hours, labels = hours)
    plt.ylabel('Value')
    plt.legend(bbox_to_anchor = (1,1))
    plt.plot()
    return

In [797]:
def grapher_kde(f, plot_dfs, group, hour):
    graph = pd.DataFrame()
    groups = {1.0:'DCI+', 0.0:'DCI-', 'all':'all'}
    
    for client in clients:
        df = plot_dfs[f][client]
        df = df.loc[[hour]]
        if group != 'all':
            df_ = df.loc[:,outcomes[client].index[outcomes[client]['label'] == group]]
        else:
            df_ = df 
        
        df_T = df_.T
        df_T.columns = [client]
        df_T.reset_index(inplace = True)
        df_T.drop('Shopid', axis = 1, inplace = True)
        
        sns.kdeplot(x = df_T[client], label = client, common_norm = False)
        
        
        '''
        graph = pd.concat([graph, df_T], axis = 1)
        df_['site'] = client
        df_ = df_.reset_index().melt(id_vars = 'site').drop('Shopid', axis = 1).iloc[1:]
        graph = pd.concat([graph, df_])
    
    graph = graph[graph['value'].isna() == False]
    graph = graph.reset_index(drop = True)
    sns.kdeplot(x = graph['value'], hue = graph['site'], hue_order = clients, common_norm = False)
    '''
    
    
    x = f[0].strip('(').strip(')')
    x = x + f'_{f[1]}'
    plt.legend(labels = clients)
    plt.title(f'Distribution_for_{x}_{groups[group]}_hours: {hour}')
    plt.show()

    